In [ ]:
import requests
import boto3

# Use API key to obtain a temporary access token (for authentication & authorization with Curation API routes)

api_key = open("../path/to/key_file").read().strip()  # SUBSTITUTE VALUE HERE -- path to file containing your API key
access_token_headers = {"x-api-key": api_key}
access_token_url = "https://api.cellxgene.dev.single-cell.czi.technology/curation/v1/auth/token"
resp = requests.post(access_token_url, headers=access_token_headers)
access_token = resp.json().get("access_token")

In [ ]:
# Use access token to obtain temporary s3 write credentials for a given Collection; these credentials will only work for THIS Collection.

collection_id = "<collection uuid>"  # SUBSTITUTE VALUE HERE -- your (non-public) Collection id, e.g. "1234abcd-5678-efab-1a2b-3c4d5e6f8a9b"
s3_credentials_url = f"https://api.cellxgene.dev.single-cell.czi.technology/curation/v1/collections/{collection_id}/datasets/s3-upload-credentials"
s3_cred_headers = {"Authorization": f"Bearer {access_token}"}
resp = requests.post(s3_credentials_url, headers=s3_cred_headers)
s3_creds = resp.json().get("Credentials")

In [ ]:
# Upload file using temporary s3 credentials

access_key_id, secret_access_key, session_token = s3_creds["AccessKeyId"], s3_creds["SecretAccessKey"], s3_creds["SessionToken"]
session = boto3.Session(
    aws_session_token=session_token,
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
)

s3 = session.client("s3")
filename = "/fully/qualified/path/to/datafile"  # SUBSTITUTE VALUE HERE -- Designate your datafile to upload; fully-qualified path
curator_tag = "arbitrary/curator_tag.h5ad"  # SUBSTITUTE VALUE HERE -- **MUST POSSESS '.h5ad' SUFFIX**; will serve as unique identifier for the resultant Dataset *within* this Collection
try:
    success = s3.upload_file(
        Filename=filename,
        Bucket="cellxgene-dataset-submissions-dev",
        Key=f"{collection_id}/{curator_tag}",
    )
except Exception as e:
    print(e)
else:
    print("File successfully uploaded!")